### DTS201TC 
#### individual Project

In [1]:
import numpy as np
import scipy.io
import scipy.stats

#### Auxiliary functions

In [2]:
def log_normpdf(x, mu, sigma):
    """
      Computes the natural logarithm of the normal probability density function
  
    """
    prob = scipy.stats.norm(mu, sigma).logpdf(x)
    return prob


In [3]:
class JointParts():
    def __init__(self, N_class):
        self.means = np.zeros([3,N_class])
        self.sigma = np.zeros([3,N_class])


In [4]:
class Model():
    def __init__(self, N_class, N_joints):
        self.jointparts = [JointParts(N_class) for i in range(N_joints)]
        self.class_priors = {}
        

In [5]:
def cal_acc(a,b):
    """
       Input
           two vectors with same size

       Output
           accuracy
    """
    
    acc = 0.0
    for i in range(len(a)):
        if(a[i]==b[i]):
            acc+=1
    acc = acc/len(a)
    
    return acc

#### Functions to implement

In [6]:
def fit_model(X):
    """
      Compute the mean and variance of X, 
    """
    # TODO
    mean = np.mean(X, axis=1)   # axis=1: Since the label is transposed in 'learn_model' function, the mean and std need to be calculated based on each row
    variance = np.std(X, axis=1)

    return (mean, variance)

In [7]:
def learn_model(dataset, labels, G=None):
    """
    Input:
     dataset: The data as loaded 
     labels:  The labels as loaded 
     
    Output: the model
     a (tentative) structure for the output model is:
       model.class_priors: containing a vector with the prior estimations
                           for each class
       model.jointparts[i] contains the estimated parameters for the i-th joint

            model.jointparts(i).means: a matrix of 3 x #classes with the
                   estimated means for each of the x,y,z variables of the 
                   i-th joint and for each class.
            model.jointparts(i).sigma: a matrix of 3 x #classes with the
                   estimated stadard deviations for each of the x,y,z
                   variables of the i-th joint and for each class.

    """

    # TODO
    # Initialization
    class_label, class_count = np.unique(labels,return_counts=True)
    N_class = len(class_label)
    [N_joints, N_pos, N_ins] = dataset.shape
    model = Model(N_class,N_joints)

    # Set prior
    class_name = [i for i in class_label]
    for i, item in enumerate(class_name):
        prior = class_count / len(labels)
        model.class_priors[item] = prior[i]

    # Set jointparts
    for i in range(N_joints):  # [0,1,2,...,19]
        joint_data = dataset[i,:,:].reshape(N_pos, N_ins)
        for j in range(N_class):  # [0,1,2,3]
            mask = (np.transpose(labels) == class_label[j])
            mask = np.tile(mask, (N_pos,1))
            class_data = joint_data[mask].reshape(N_pos,-1)
            model.jointparts[i].means[:,j], model.jointparts[i].sigma[:,j] = fit_model(class_data)

    return model

In [8]:
def classify_samples(instances, model):
    """    
    Input
       instance: a 20x3x#instances matrix defining body positions of
                 instances
       model: as the output of learn_model

    Output
       probs: a matrix of #instances x #classes with the probability of each
              instance of belonging to each of the classes

    Important: to avoid underflow numerical issues this computations should
               be performed in log space
    """
    # TODO
#     probs = likelihood * priors


    # Initialization
    prob_class = np.zeros((1,len(model.class_priors)))              # One instance's probability of 4 classes: [1,4]
    probs = np.zeros((instances.shape[2],len(model.class_priors)))  # All instances' probability of 4 classes: [120,4]

    # Compute the posterior probability for each instance belonging to each class
    for instance_idx in range(instances.shape[2]):
        instances_each = instances[:,:,instance_idx].reshape(instances.shape[0],instances.shape[1])
        for class_idx in range(len(model.class_priors)):
            prior_keys = list(model.class_priors.keys())
            temp_sum = np.log(model.class_priors[prior_keys[class_idx]])
            for joint_idx in range(len(model.jointparts)):
                mu_class = model.jointparts[joint_idx].means[:,class_idx]
                sigma_class = model.jointparts[joint_idx].sigma[:,class_idx]
                temp_sum += sum(log_normpdf(instances_each[joint_idx,:], mu_class, sigma_class))
            prob_class[:,class_idx] = temp_sum
        probs[instance_idx,:] = prob_class


    return probs


# TEST PART

## 1 learn model

In [9]:
dd = scipy.io.loadmat('data/validation_data.mat')
X = dd['data_small']
Y = dd['labels_small']
# the index for training set
ind = dd['train_indexes']
[trainInd,nouse] = np.where(ind==1)
# get the training set with fixed index
X_train = X[:,:,trainInd]
y_train = Y[trainInd]
# training/learning
modelNB = learn_model(X_train, y_train)

## 2 Classify

In [10]:
dd = scipy.io.loadmat('data/validation_data.mat')
X = dd['data_small']
Y = dd['labels_small']

# get the test set index
ind = dd['test_indexes']
[testInd,nouse] = np.where(ind==1)
# get the testing set with fixed index
X_test = X[:,:,testInd]
y_test = Y[testInd]
# classify  
y_rst = classify_samples(X_test, modelNB)
# print(y_rst)
labelpos = np.argmax(y_rst, axis = 1)

# get the label of the each instance with probs
classPriors = modelNB.class_priors
whatlabel = list(classPriors.keys())
y = classify_samples(X_test, modelNB)
labelpos = np.argmax(y, axis = 1)
rst = [whatlabel[labelpos[i]] for i in range(len(labelpos))]
val = list(y_test.reshape(len(y_test),))
print(cal_acc(rst,val))

0.95
